##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Перенос стиля с помощью нейронной сети

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/generative/style_transfer"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/style_transfer.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/generative/style_transfer.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
      <a href="https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">Смотрите модель на TF Hub</a>
    </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/generative/style_transfer.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

В этом уроке используется глубокое обучение для преобразования одного изображения в стиль другого изображения (когда-нибудь хотелось рисовать как Пикассо или Ван Гог?). Это способ известен как *передача нейронного стиля*, и его методика описана в <a href="https://arxiv.org/abs/1508.06576" class="external"> нейронном алгоритме художественного стиля </a> (Gatys и другие).

Примечание. В этом руководстве демонстрируется оригинальный алгоритм передачи стилей. Он преобразует контент изображения под определенный стиль. Современные подходы учат модель генерировать стилизованное изображение напрямую(аналогично [cyclegan](cyclegan.ipynb)). Этот подход намного быстрее(до 1000 раз).

Для простого применения переноса стилей ознакомьтесь с этим [учебным пособием](https://www.tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization), чтобы узнать больше о том, как использовать предварительно обученную [модель произвольной стилизации изображения](https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2) из [TensorFlow Hub](https://tfhub.dev) или как использовать модель передачи стилей из [TensorFlow Lite](https://www.tensorflow.org/lite/models/style_transfer/overview).

Перенос нейронного стиля - это метод оптимизации, используемый для получения двух изображений - изображения *содержимого* и изображения *эталона стиля* (например, работы известного художника) - и смешивания их вместе так, чтобы выходное изображение выглядело как входное, но «нарисованое» в стиле эталонного изображения.

Это реализуется путем оптимизации выходного изображения для соответствия данным контента входного изображения и данным стиля эталонного изображения. Эти данные извлекается из изображений с помощью сверточной сети.

Например, возьмем это изображение собаки и Композицию 7 Василия Кандинского:

<img src="https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg" width="500px"/>

[Желтый лабрадор](https://commons.wikimedia.org/wiki/File:YellowLabradorLooking_new.jpg), из [Wikimedia](https://en.wikipedia.org/wiki/User:Elf). Лицензия [CC BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0/deed.en)

<img src="https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg" width="500px"/>


А как бы это выглядело, если бы Кандинский решил нарисовать эту собаку в своем стиле? Что-то вроде этого?

<img src="https://tensorflow.org/tutorials/generative/images/stylized-image.png" style="width: 500px;"/>

## Установка


### Импорт и настройка модулей

In [ ]:
import tensorflow as tf

In [ ]:
import IPython.display as display

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

import numpy as np
import PIL.Image
import time
import functools

In [ ]:
def tensor_to_image(tensor):
  tensor = tensor*255
  tensor = np.array(tensor, dtype=np.uint8)
  if np.ndim(tensor)>3:
    assert tensor.shape[0] == 1
    tensor = tensor[0]
  return PIL.Image.fromarray(tensor)

Загрузите изображения и выберите изображение стиля и изображение содержимого:

In [ ]:
content_path = tf.keras.utils.get_file('YellowLabradorLooking_new.jpg', 'https://storage.googleapis.com/download.tensorflow.org/example_images/YellowLabradorLooking_new.jpg')
style_path = tf.keras.utils.get_file('kandinsky5.jpg','https://storage.googleapis.com/download.tensorflow.org/example_images/Vassily_Kandinsky%2C_1913_-_Composition_7.jpg')

## Визуализация ввода

Определите функцию для загрузки изображения и ограничьте его максимальный размер до 512 пикселей.

In [ ]:
def load_img(path_to_img):
  max_dim = 512
  img = tf.io.read_file(path_to_img)
  img = tf.image.decode_image(img, channels=3)
  img = tf.image.convert_image_dtype(img, tf.float32)

  shape = tf.cast(tf.shape(img)[:-1], tf.float32)
  long_dim = max(shape)
  scale = max_dim / long_dim

  new_shape = tf.cast(shape * scale, tf.int32)

  img = tf.image.resize(img, new_shape)
  img = img[tf.newaxis, :]
  return img

Создайте простую функцию для показа изображения:

In [ ]:
def imshow(image, title=None):
  if len(image.shape) > 3:
    image = tf.squeeze(image, axis=0)

  plt.imshow(image)
  if title:
    plt.title(title)

In [ ]:
content_image = load_img(content_path)
style_image = load_img(style_path)

plt.subplot(1, 2, 1)
imshow(content_image, 'Content Image')

plt.subplot(1, 2, 2)
imshow(style_image, 'Style Image')

## Быстрая передача стилей с использованием TF-Hub

В этом руководстве демонстрируется оригинальный алгоритм передачи стилей, который оптимизирует содержимое изображения в соответствии с определенным стилем. Прежде чем вдаваться в подробности, давайте посмотрим, как [модель TensorFlow Hub](https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2) делает это:

In [ ]:
import tensorflow_hub as hub
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')
stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]
tensor_to_image(stylized_image)

## Определение представлений содержимого и стиля

Используйте промежуточные слои модели, чтобы получить представление *содержимого* и *стиля* изображения. Начиная с входного слоя сети, первые несколько активаций слоя представляют низкоуровневые функции, такие как края и текстуры. По мере того, как вы проходите по сети, последние несколько слоев представляют элементы более высокого уровня - части объектов, такие как *колеса* или *глаза*. В этом руководстве вы используете сетевую архитектуру VGG19, предварительно обученную сеть для классификации изображений. Эти промежуточные слои необходимы для определения представления содержимого и стиля изображений. Для входного изображения попробуйте сопоставить соответствующие представления стиля и содержимого в этих промежуточных слоях.


Загрузите [VGG19](https://keras.io/applications/#vgg19) и протестируйте сеть на нашем изображении, чтобы убедиться, что она используется правильно:

In [ ]:
x = tf.keras.applications.vgg19.preprocess_input(content_image*255)
x = tf.image.resize(x, (224, 224))
vgg = tf.keras.applications.VGG19(include_top=True, weights='imagenet')
prediction_probabilities = vgg(x)
prediction_probabilities.shape

In [ ]:
predicted_top_5 = tf.keras.applications.vgg19.decode_predictions(prediction_probabilities.numpy())[0]
[(class_name, prob) for (number, class_name, prob) in predicted_top_5]

Теперь загрузите `VGG19` без верхних слоев и просмотрите имена слоев.

In [ ]:
vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')

print()
for layer in vgg.layers:
  print(layer.name)

Выберите промежуточные слои сети для представления стиля и содержания изображения:

In [ ]:
content_layers = ['block5_conv2'] 

style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1', 
                'block4_conv1', 
                'block5_conv1']

num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

#### Промежуточные слои для стиля и контента изображения

Так почему же эти промежуточные результаты в нашей предварительно обученной сети классификации изображений позволяют нам определять представления для стиля и контента?

На высоком уровне, чтобы сеть могла выполнять классификацию изображений(то, для чего эта сеть была обучена), она должна понимать изображение. Для этого требуется взять необработанное изображение в виде входных пикселей и построить внутреннее представление, которое преобразует необработанные пиксели изображения в комплексное понимание признаков, присутствующих в изображении.

Это одна из причин того, почему сверточные нейронные сети способны хорошо обобщать: они способны улавливать инвариативность и определять особенности внутри классов(например, кошки против собак), которые не зависят от фонового шума и других погрешностей. Таким образом, где-то между загрузкой необработанного изображения в модель, и меткой классификации выходных данных, модель занимается комплексным извлечением определяющих признаков. Получив доступ к промежуточным слоям модели, вы можете описать содержимое и стиль входных изображений.

## Создание модели 

Сети в `tf.keras.applications` спроектированы таким образом, чтобы вы могли легко извлекать значения промежуточного слоя с помощью функционального API Keras.

Чтобы определить модель с помощью функционального API, укажите входы и выходы:

`model = Model(inputs, outputs)`

Следующая функция создает модель VGG19, которая возвращает список выходных данных промежуточного уровня:

In [ ]:
def vgg_layers(layer_names):
  """ Создание модели vgg, которая возвращает список промежуточных выходных значений."""
  # Загружаем нашу модель. Загружаем предварительно обученную на imagenet данных VGG сеть
  vgg = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
  vgg.trainable = False
  
  outputs = [vgg.get_layer(name).output for name in layer_names]

  model = tf.keras.Model([vgg.input], outputs)
  return model

И создайте модель:

In [ ]:
style_extractor = vgg_layers(style_layers)
style_outputs = style_extractor(style_image*255)

# Смотрим статистику вывода каждого слоя
for name, output in zip(style_layers, style_outputs):
  print(name)
  print("  shape: ", output.numpy().shape)
  print("  min: ", output.numpy().min())
  print("  max: ", output.numpy().max())
  print("  mean: ", output.numpy().mean())
  print()

## Расчет стиля

Контент изображения представлен значениями промежуточных карт признаков.

Оказывается, стиль изображения можно описать состояниями и отношениями между различными картами функций. Вычислите грамм-матрицу(Gram matrix), которая включает эту информацию, взяв внешнее произведение вектора признаков с самим собой в каждом месте и усреднив этот внешний произведение по всем ячейкам. Эта грамм-матрица может быть рассчитана для конкретного слоя как:

$$G^l_{cd} = \frac{\sum_{ij} F^l_{ijc}(x)F^l_{ijd}(x)}{IJ}$$

Кратко это можно реализовать с помощью функции `tf.linalg.einsum`:

In [ ]:
def gram_matrix(input_tensor):
  result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
  input_shape = tf.shape(input_tensor)
  num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
  return result/(num_locations)

## Извлечение стиля и контента


Создайте модель, которая возвращает тензоры стиля и контента.

In [ ]:
class StyleContentModel(tf.keras.models.Model):
  def __init__(self, style_layers, content_layers):
    super(StyleContentModel, self).__init__()
    self.vgg =  vgg_layers(style_layers + content_layers)
    self.style_layers = style_layers
    self.content_layers = content_layers
    self.num_style_layers = len(style_layers)
    self.vgg.trainable = False

  def call(self, inputs):
    "Ожидается ввод типа float в диапазоне [0,1]"
    inputs = inputs*255.0
    preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
    outputs = self.vgg(preprocessed_input)
    style_outputs, content_outputs = (outputs[:self.num_style_layers], 
                                      outputs[self.num_style_layers:])

    style_outputs = [gram_matrix(style_output)
                     for style_output in style_outputs]

    content_dict = {content_name:value 
                    for content_name, value 
                    in zip(self.content_layers, content_outputs)}

    style_dict = {style_name:value
                  for style_name, value
                  in zip(self.style_layers, style_outputs)}
    
    return {'content':content_dict, 'style':style_dict}

При вызове на изображении эта модель возвращает грамм-матрицу(стиль) `style_layers` и содержимое `content_layers`:

In [ ]:
extractor = StyleContentModel(style_layers, content_layers)

results = extractor(tf.constant(content_image))

print('Styles:')
for name, output in sorted(results['style'].items()):
  print("  ", name)
  print("    shape: ", output.numpy().shape)
  print("    min: ", output.numpy().min())
  print("    max: ", output.numpy().max())
  print("    mean: ", output.numpy().mean())
  print()

print("Contents:")
for name, output in sorted(results['content'].items()):
  print("  ", name)
  print("    shape: ", output.numpy().shape)
  print("    min: ", output.numpy().min())
  print("    max: ", output.numpy().max())
  print("    mean: ", output.numpy().mean())


## Запуск градиентного спуска

С помощью этого средства извлечения стилей и содержимого теперь можно реализовать алгоритм передачи стилей. Сделайте это, вычислив среднеквадратическую ошибку для вывода вашего изображения относительно каждой цели, а затем возьмите взвешенную сумму этих потерь.

Установите целевые значения стиля и содержания:

In [ ]:
style_targets = extractor(style_image)['style']
content_targets = extractor(content_image)['content']

Определите `tf.Variable`, которая будет содержать изображение для оптимизации. Чтобы сделать это быстро, инициализируйте его контентом изображения(`tf.Variable` должна иметь ту же размерность, что и контент изображения):

In [ ]:
image = tf.Variable(content_image)

Поскольку это изображение состоит из чисел типа float, определите функцию для сохранения значений пикселей между 0 и 1:

In [ ]:
def clip_0_1(image):
  return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

Создайте оптимайзер. В статье рекомендуется LBFGS, но `Adam` тоже работает хорошо:

In [ ]:
opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

Чтобы оптимизировать модель, используйте взвешенную комбинацию двух потерь, чтобы получить общую величину потерь:

In [ ]:
style_weight=1e-2
content_weight=1e4

In [ ]:
def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                           for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers

    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                             for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss

Используйте `tf.GradientTape` для обновления изображения.

In [ ]:
@tf.function()
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

Теперь выполните несколько шагов для проверки:

In [ ]:
train_step(image)
train_step(image)
train_step(image)
tensor_to_image(image)

Поскольку все работает, выполните более длительную оптимизацию:

In [ ]:
import time
start = time.time()

epochs = 10
steps_per_epoch = 100

step = 0
for n in range(epochs):
  for m in range(steps_per_epoch):
    step += 1
    train_step(image)
    print(".", end='')
  display.clear_output(wait=True)
  display.display(tensor_to_image(image))
  print("Train step: {}".format(step))
  
end = time.time()
print("Total time: {:.1f}".format(end-start))

## Общее значение потерь

Одним из недостатков этой базовой реализации является то, что она создает множество высокочастотных артефактов. Уменьшите их, используя явный член регуляризации высокочастотных компонентов изображения. В переносе стилей это часто называют *общими значениями потерь*:

In [ ]:
def high_pass_x_y(image):
  x_var = image[:,:,1:,:] - image[:,:,:-1,:]
  y_var = image[:,1:,:,:] - image[:,:-1,:,:]

  return x_var, y_var

In [ ]:
x_deltas, y_deltas = high_pass_x_y(content_image)

plt.figure(figsize=(14,10))
plt.subplot(2,2,1)
imshow(clip_0_1(2*y_deltas+0.5), "Horizontal Deltas: Original")

plt.subplot(2,2,2)
imshow(clip_0_1(2*x_deltas+0.5), "Vertical Deltas: Original")

x_deltas, y_deltas = high_pass_x_y(image)

plt.subplot(2,2,3)
imshow(clip_0_1(2*y_deltas+0.5), "Horizontal Deltas: Styled")

plt.subplot(2,2,4)
imshow(clip_0_1(2*x_deltas+0.5), "Vertical Deltas: Styled")

Код выше показывает, как увеличились высокочастотные составляющие.
Кроме того, этот высокочастотный компонент в основном является детектором граней. Вы можете получить аналогичный результат от детектора граней Собеля(Sobel edge detector), например:

In [ ]:
plt.figure(figsize=(14,10))

sobel = tf.image.sobel_edges(content_image)
plt.subplot(1,2,1)
imshow(clip_0_1(sobel[...,0]/4+0.5), "Horizontal Sobel-edges")
plt.subplot(1,2,2)
imshow(clip_0_1(sobel[...,1]/4+0.5), "Vertical Sobel-edges")

Связанная с этим потеря регуляризации представляет собой сумму квадратов значений:

In [ ]:
def total_variation_loss(image):
  x_deltas, y_deltas = high_pass_x_y(image)
  return tf.reduce_sum(tf.abs(x_deltas)) + tf.reduce_sum(tf.abs(y_deltas))

In [ ]:
total_variation_loss(image).numpy()

Код выше демонстрирует внутреннюю реализацию расчета потерь регуляризации. Но нет необходимости реализовывать это самостоятельно, TensorFlow включает стандартную реализацию:

In [ ]:
tf.image.total_variation(image).numpy()

## Перезапуск оптимизации

Выберите вес для `total_variation_loss`:

In [ ]:
total_variation_weight=30

Теперь включите его в функцию `train_step`:

In [ ]:
@tf.function()
def train_step(image):
  with tf.GradientTape() as tape:
    outputs = extractor(image)
    loss = style_content_loss(outputs)
    loss += total_variation_weight*tf.image.total_variation(image)

  grad = tape.gradient(loss, image)
  opt.apply_gradients([(grad, image)])
  image.assign(clip_0_1(image))

Повторно инициализируйте переменную оптимизации:

In [ ]:
image = tf.Variable(content_image)

И запустите оптимизацию:

In [ ]:
import time
start = time.time()

epochs = 10
steps_per_epoch = 100

step = 0
for n in range(epochs):
  for m in range(steps_per_epoch):
    step += 1
    train_step(image)
    print(".", end='')
  display.clear_output(wait=True)
  display.display(tensor_to_image(image))
  print("Train step: {}".format(step))

end = time.time()
print("Total time: {:.1f}".format(end-start))

Наконец, сохраните результат:

In [ ]:
file_name = 'stylized-image.png'
tensor_to_image(image).save(file_name)

try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(file_name)

## Следующие шаги

В этом руководстве демонстрируется оригинальный алгоритм передачи стилей. Для простого применения переноса стиля просмотрите этот [учебник](https://www.tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization), чтобы узнать больше о том, как использовать модель переноса произвольного стиля изображения из [TensorFlow Hub](https://tfhub.dev).